# Manipulate Neuroimaging Data with Python

This notebook is a revision of the excellent [Dartbrains Introductory Notebook](https://dartbrains.org/content/Introduction_to_Neuroimaging_Data.html).

**TIP**: Some of these cells are slow to run! To determine whether a cell has been run, look at the square brackets on the left:
- `[ ]` indicates the cell has not been run
- `[*]` indicates the cell is running.  Be patient!
- A number in the brackets, e.g., `[35]`, indicates that the cell has been run, and you can move on to the next one. 

---
# Neuroimaging Libraries

Many Python neuroimaging libraries exist and more are being developed.    
In this notebook, you will explore the following libraries:   
- [PyBIDS](https://bids-standard.github.io/pybids/), a library for working with Brain Imaging Data Structure BIDS data
- [NiBabel](https://nipy.org/packages/nilearn/index.html), Reads and writes common neuroimaging file formats.
- [nilearn](https://nipy.org/packages/nilearn/index.html), Fast and easy statistical learning on neuroimaging data.
- [NLTools](https://nltools.org/auto_examples/index.html), For analyzing neuroimaging data, including tools to perform data manipulation and analyses such as univariate GLMs, predictive multivariate modeling, and representational similarity analyses.

---
# Get the Data

Download the data we will be using from [OSF](https://osf.io/5q3m8) with *wget*, unzip it and clean up the zip file.

In [ ]:
import os
import wget
import zipfile

site_url = 'https://osf.io/5q3m8/download'

if (not os.path.isdir('data')):
    wget.download(site_url)
    with zipfile.ZipFile('Jupyter_neuro_data.zip','r') as zip_ref: 
        zip_ref.extractall(path=None)

In [ ]:
if os.path.exists('Jupyter_neuro_data.zip'):
   os.remove('Jupyter_neuro_data.zip')

---

# NiBabel

Neuroimaging data is often stored in the format of NIfTI files `.nii` which can also be compressed using gzip `.nii.gz`.  These files store both 3D and 4D data and also contain structured metadata in the image **header**.

There is an very nice tool to access NIfTI data stored on your file system in Python called [NiBabel](http://nipy.org/nibabel/).

NiBabel objects can be initialized by specifying a NIfTI file, and you can specify the NIfTI file using BIDSLayout!  

First, import the NiBabel module as `nib` (short and sweet so you don't have to type so much when using the tool).     
Include a path to the data file so you don't have to constantly type this.     

Below, we load an anatomical image (`nib.load`) from subject 219, using PyBIDS )`layout.get`) to specify the image.

In [ ]:
import nibabel as nib

# The following specifies the path to the data using BIDSLayout
T1w_data = nib.load(layout.get(subject='219', session='itbs', suffix='T1w', return_type='file', extension='nii.gz')[0])


## Get Information about the NiBabel Object
To get more help on working with NiBabel data objects, either consult the [documentation](https://nipy.org/nibabel/tutorials.html#tutorials) or add a `?`.

In [ ]:
T1w_data?

## Get the Object Dimensions with NiBabel
The imaging data is stored in either a 3D or 4D numpy array. Just like numpy, it is easy to get the dimensions of the data using `shape`. 

In [ ]:
T1w_data.shape

## Plot a Slice of the Data Object with matplotlib
Looks like there are 3 dimensions (x,y,z) that is the number of voxels in each dimension. If we know the voxel size, we could convert this into millimeters.

We can also directly access the data and plot a single slice using standard matplotlib functions. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.imshow(T1w_data.get_fdata()[:,:,100])

## Plot Different Slices
Try slicing different dimensions (x,y,z) to get a feel for how the data is represented in this anatomical image. You can do this by changing the values in the square brackets, e.g., try [100,:,:]

## Look into the Image Header with NiBabel
An image consists of two parts, the data and the header.  The header contains metadata. We can also access data from the image header. Let's assign the header of an image to a variable and print it to view it's contents.

In [ ]:
header = T1w_data.header
print(header)      

## Get the Affine Matrix with NiBabel
Some of the important information in the header is information about the orientation of the image in space. This can be represented as the affine matrix, which can be used to transform images between different spaces.

In [ ]:
T1w_data.affine

## Summary of NiBabel
NiBabel is used to load images into Python. Once ther images are loaded, you can learn more about them by examining their headers. In addition, you can manipulate the images with Python libraries like matplotlib. Let's look at some other Python libraies for neuroimaging.

---
# Nilearn

There are many useful tools from the [nilearn](https://nilearn.github.io/index.html) library to help manipulate and visualize neuroimaging data. See their [documentation](https://nilearn.github.io/plotting/index.html#different-plotting-functions) for examples.

This section explores a few of their different plotting functions, which can work directly with NiBabel instances.

Some of these functions are SLOW.  If the cell is marked like `[*]` then wait till it finishes running and gets assigned a number!

In [ ]:
%matplotlib inline

from nilearn.plotting import view_img, plot_glass_brain, plot_anat, plot_epi

## plot_anat

In [ ]:
# This is a slow one!
plot_anat(T1w_data)

Nilearn plotting functions are very flexible and allow us to easily customize our plots.   

In [ ]:
plot_anat(T1w_data, draw_cross=False, display_mode='z')

Try changing the display_mode to x or y.

## Get More Information about plot_anat
Get more information about how to use the function with `?` and try to add different commands to change the plot.

nilearn also has a neat interactive viewer called `view_img` for examining images directly in the notebook. 

In [ ]:
plot_anat?

## view_img

In [ ]:
view_img(T1w_data)

The `view_img` function is particularly useful for overlaying statistical maps on an anatomical image to interactively examine where the results are located.

As an example, load a mask of the amygdala and try to find where it is located. Download it from [Neurovault](https://neurovault.org/images/18632/) using a function from NLTools.

In [ ]:
from nltools.data import Brain_Data
amygdala_mask = Brain_Data('https://neurovault.org/media/images/1290/FSL_BAmyg_thr0.nii.gz').to_nifti()

view_img(amygdala_mask, T1w_data)

## Glass Brain
Plot a glass brain which allows us to see through the brain from different slice orientations. In this example, we will plot the binary amygdala mask.

In [ ]:
plot_glass_brain(amygdala_mask)

---
# NLTools (a.k.a Neurolearn)

You've learned how to use NiBabel to load imaging data and nilearn to plot it.

The NLTools package tries to make loading, plotting, and manipulating data easier. It uses many functions from NiBabel, nilearn, and other Python libraries. The bulk of the NLTools toolbox is built around the `Brain_Data()` class. The concept behind the class is to have a similar feel to a pandas dataframe, which means that it should feel intuitive to manipulate the data.

The `Brain_Data()` class has several attributes that may be helpful to know about. First, it stores imaging data in `.data` as "vectorized features x observations matrix". 

- Each image is an observation and each voxel is a feature. 
- Space is flattened using `nifti_masker` from nilearn. 
- This object is also stored as an attribute in `.nifti_masker` to allow transformations from 2D to 3D/4D matrices. 
- In addition, a brain_mask is stored in `.mask`. 
- Finally, there are attributes to store either class labels for prediction/classification analyses in `.Y` and design matrices in `.X`. These are both expected to be pandas `DataFrames`.

We will give a quick overview of basic Brain_Data operations, but we encourage you to see our [documentation](https://nltools.org/) for more details.

## Brain_Data basics
To get a feel for the `Brain_Data` class, load an example anatomical overlay image that comes packaged with the toolbox.

In [ ]:
from nltools.data import Brain_Data
from nltools.utils import get_anatomical

anat = Brain_Data(get_anatomical())
anat

### Brain_Data View Attributes
To view the attributes of `Brain_Data` use the `vars()` function.

In [ ]:
print(vars(anat))

`Brain_Data` has many methods to help manipulate, plot, and analyze imaging data. We can use the `dir()` function to get a quick list of all of the available methods that can be used on this class.

To learn more about how to use these tools either use the `?` function, or look up the function in the [api documentation](https://nltools.org/api.html).


In [ ]:
print(dir(anat))

Ok, now let's load a single subject's functional data from the run 1 resting state dataset. We will load one that has already been preprocessed with fmriprep and is stored in the derivatives folder.

### Load Data
Loading data can be **slow** especially if the data need to be resampled to the template, which is set at $2mm^3$ by default. However, once it's loaded into the workspace it should be relatively fast to work with it.


In [ ]:
sub = 'sub-219'
ses = 'ses-itbs'

fmr_data = Brain_Data(os.path.join(data_dir, 'derivatives', 'fmriprep', sub, ses, 'func', f'{sub}_{ses}_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'))

Here are a few quick basic data operations.

### Print Information about the Dimensions of the Dataset 
Find the number of image volumes in the Brain_Data() instance. This example is a 4D fMRI image with 177 volumes. 

In [ ]:
print(len(fmr_data))

Find the dimensions of Brain_Data (images x voxels).  The number of voxels from each volume included in the Brain_Data is determined by the brain mask (the 2mm MNI space brain mask you loaded earlier).

In [ ]:
print(fmr_data.shape())

### Learn more about Brain_Data

In [ ]:
Brain_Data?

### Copy the image data
You can copy the Brain_Data object

In [ ]:
fmr_data2 = fmr_data.copy()

### Apply mathematical operations
Alternatively, you can make a modified copy of the data. This example adds 10 to each voxel and multiplies by 2.
Note that most neuroimaging packages contain these sorts of element-by-element mathematical operations.

In [ ]:
fmr_data2 = (fmr_data + 10) * 2

### Remove the linear trend from every voxel
Create another dataset with detrended time series for each voxel 

In [ ]:
fmr_detrend = fmr_data.detrend()
# Return the datatype
fmr_detrend.dtype()

### List the Current variables
Use the iPython magic command [whos](https://iPython.readthedocs.io/en/stable/interactive/magics.html?highlight=%25whos#magic-whos)

In [ ]:
%whos

### Plotting examples
Whether you plot the `mean` or `std`, note how the range of the y-axis changes if you plot `fmr_data` vs `fmr_data2` vs `fmr_detrend`.  Change the examples to plot std instead.

In [ ]:
plt.plot(fmr_data.mean(axis=1))

In [ ]:
# fmr_data2 had 10 added to each voxel and was then doubled
plt.plot(fmr_data2.mean(axis=1))

In [ ]:
plt.plot(fmr_detrend.mean(axis=1))

###  Concatenate Brain_Data
We have several Brain_Data objects now.  It is possible to concatenate them.

In [ ]:
fmr_concat=fmr_data.append(fmr_data2)

In [ ]:
# Print the length of the new Brain_Data object to see that it is indeed 2*177
print(len(fmr_concat))

### Brain_Data to NIfTI
Brain_Data instances can be easily converted to NiBabel instances, which store the data in a 3D/4D matrix.  This is useful for interfacing with other Python toolboxes such as [nilearn](http://nilearn.github.io)

In [ ]:
fmr_data.to_nifti()

Brain_Data objects can also be written out to disk as NIfTI images.

In [ ]:
fmr_data.write('data/outputs/Tmp_Data.nii.gz')

Lists of `Brain_Data` instances can also be concatenated by recasting as a `Brain_Data` object.

In [ ]:
print(type([x for x in fmr_data[:4]]))

type(Brain_Data([x for x in fmr_data[:4]]))

Images within a Brain_Data() instance are iterable.  Here we use a list comprehension to calculate the overall mean across all voxels within an image.

In [ ]:
[x.mean() for x in fmr_data]

Though, we could also do this with the `mean` method by setting `axis=1`.

In [ ]:
fmr_data.mean(axis=1)

Let's plot the mean to see how the global signal changes over time.

In [ ]:
plt.plot(fmr_data.mean(axis=1))

Notice the slow linear drift over time, where the global signal intensity gradually decreases. We will learn how to remove this with a high pass filter in future tutorials.

### Plotting
There are multiple ways to plot your data.

For a very quick plot, you can return a montage of axial slices with the `.plot()` method. As an example, we will plot the mean of each voxel over time.

In [ ]:
f = fmr_data.mean().plot()

Brain_Data() instances can be converted to a NiBabel instance and plotted using any nilearn plot method such as glass brain.


In [ ]:
plot_glass_brain(fmr_data.mean().to_nifti())

Ok, that's the basics. `Brain_Data` can do much more!
Check out some of our [tutorials](https://nltools.org/auto_examples/index.html) for more detailed examples.
